# Testing Environment

This notebook can be used to compare a custom Prompto chatbot against target answers specified in the q_and_a_gen.json.

In [26]:
!pip install langchain openai jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 4.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [10]:
import json
from dotenv import load_dotenv

load_dotenv()

True

In [45]:
demo_json_file_path = "demo_help.json"

In [46]:
with open(demo_json_file_path, "r") as demo_file:
    demo_json = json.load(demo_file)

In [49]:
demo_docs = []

In [50]:
from langchain_core.documents import Document

for entry in demo_json["data"]:
    demo_docs.append(Document(page_content=entry["question"] + "\n\n" + entry["ai_answer"],
                                metadata={'source': demo_json_file_path}))

In [52]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores.chroma import Chroma

demo_db = Chroma.from_documents(demo_docs, OpenAIEmbeddings())

demo_retriever = demo_db.as_retriever()

In [53]:
demo_retriever.get_relevant_documents("How can I create a movable player?")

[Document(page_content='How can I create a movable player?\n\nTo create a movable player you need to: 1. Create a CharacterBody2D. 2. Add a PlatformerControllerBehavior as a child. 3. Add a PlaceholderBehavior as a child.', metadata={'source': 'demo_help.json'}),
 Document(page_content='How can I create moving platforms?\n\nTo create moving platforms for the player to jump on you need to: 1. Create a Path2D. 2. Add a PathFollow2D as a child of the Path2D. 3. Add a CharacterBody2D as a child of the PathFollow2D. 4. Add a PlaceholderBehavior as a child of the CharacterBody2D 5. Add an AlwaysBehavior as a child of the CharacterBody2D. 6. Connect the AlwaysBehavior with the PathFollow2D. 7. In the <statement> field add set_progress. 8. For updating the progress add a speed value of your choice.', metadata={'source': 'demo_help.json'}),
 Document(page_content='How can I create platforms for the player to jump on?\n\nTo create platforms for the player to jump on you need to: 1. Create a Stat

In [4]:
qa_gen_json_file_path = "q_and_a_gen.json"

In [5]:
with open(qa_gen_json_file_path, "r") as qa_json_file:
    qa_json = json.load(qa_json_file)

In [6]:
qa_data = qa_json["data"]

In [8]:
len(qa_data)

15

In [11]:
from langchain.prompts import PromptTemplate

template = "Below are two answers from questions about a game prototyping framework called Pronto that is build on top of the Godot Game Engine. The first answer is correct. The second answer should be compared against first one and checked if it answers the question. To evaluate the result, please simply reply \"Yes\" or \"No\" whether the question was answered or not.\n\nHere is the question: {question}\n\nHere is the first answer: {answer1}\n\nHere is the second answer: {answer2}"

prompt = PromptTemplate(template=template, input_variables=["question", "answer1", "answer2"])

In [12]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4")

In [13]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [14]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredMarkdownLoader

docs_loader = DirectoryLoader("pronto-docs", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader)
docs = docs_loader.load()

In [15]:
from langchain_community.chat_models import ChatOpenAI

chat_llm = ChatOpenAI(temperature=0)

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain_community.vectorstores.chroma import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.embeddings import OpenAIEmbeddings

parent_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=400, chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=100, chunk_overlap=0)

pdlg_vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
pdlg_store = InMemoryStore()
pdlg_retriever = ParentDocumentRetriever(
    vectorstore=pdlg_vectorstore,
    docstore=pdlg_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

pdlg_retriever.add_documents(docs)

In [17]:
from langchain.retrievers import MultiQueryRetriever

pdlg_mq_retriever = MultiQueryRetriever.from_llm(
    retriever=pdlg_retriever, llm=chat_llm
)

In [18]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[pdlg_mq_retriever, pdlg_retriever], weights=[0.5, 0.5]
)

In [19]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)

In [20]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(chat_llm, retriever=ensemble_retriever, memory=memory)

In [16]:
results = {}
for data in qa_data:
    question = data["question"]
    answer1 = data["ai_answer"]
    answer2 = qa(question)
    result = llm_chain.run({
        "question": question,
        "answer1": answer1,
        "answer2": answer2
    })
    results[question] = result
    print(question, result + "/5")

How do I connect two behaviors? Yes/5
How do I adjust the properties of a PlatformerControllerBehavior? Yes/5
How can I damage a player? Yes/5
What does an AlwaysBehavior do? Yes/5
How to change the sprite of a PlaceholderBehavior? Yes/5


KeyboardInterrupt: 